In [ ]:
import geopandas as gpd
from libpysal.weights import DistanceBand
from spreg import GM_Error
from spreg import GM_Lag

# import data
gdf = gpd.read_file("Geodaten100m/Buildings_LST_Points_100m_oneGroup.shp")
y = gdf[["RoofEff"]].values
X = gdf[["LST"]].values

# calculate weight matrix based on inverse distances with treshold 100 m
w = DistanceBand.from_dataframe(gdf, threshold=100)
w.transform = "r"

# calculate spatial lag modell
slm_model = GM_Lag(y, X, w=w)
data_reg_slm = []
data_reg_slm.append({
    "intercept": slm_model.betas[0][0],
    "slope": slm_model.betas[1][0],
    "pseudo_r_squared": slm_model.pr2,
    "p_value": slm_model.z_stat[1][1],
    "rho": slm_model.rho[0],
    "p_value_rho": slm_model.z_stat[2][1],
})
reg_slm = pd.DataFrame(data_reg_slm) 
print("Spatial Lag:")
print(reg_slm)
gdf["slm_Resid"] = slm_model.u.flatten()

# calculate spatial error modell
sem_model = GM_Error(y, X, w=w)
data_reg_sem = []
data_reg_sem.append({
    "intercept": sem_model.betas[0][0],
    "slope": sem_model.betas[1][0],
    "pseudo_r_squared": sem_model.pr2,
    "p_value": sem_model.z_stat[1][1],
    "lambda": sem_model.betas[-1][0],
    "p_value_lambda": sem_model.z_stat[-1][1],
})
reg_sem = pd.DataFrame(data_reg_sem) 
print("\nSpatial Error:")
print(reg_sem)
gdf["sem_Resid"] = sem_model.u.flatten()

# export shape-file with residuals
gdf.to_file("GeodatenResid\Buildings_LST_Points_100m_oneGroup_Resid.shp", driver="ESRI Shapefile", encoding="utf-8")